In [1]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau

print( tf.VERSION )


1.13.1


In [2]:
encoder_input_data = np.load( 'encoder_input_data_friends_short.npy' )
decoder_input_data = np.load( 'decoder_input_data_friends_short.npy' )
decoder_target_data = np.load( 'decoder_target_data_friends_short.npy' )

embedding_matrix = np.load('embedding_matrix_friends_short.npy' ) 

tokenizer = pickle.load( open('tokenizer_friends_short.pkl' , 'rb'))

num_tokens = len( tokenizer.word_index )+1
word_dict = tokenizer.word_index

max_question_len = encoder_input_data.shape[1]
max_answer_len = decoder_input_data.shape[1]

print( 'Max length of question is {}'.format( max_question_len) )
print( 'Max length of answer is {}'.format( max_answer_len) )

print( encoder_input_data.shape )
print( decoder_input_data.shape )
print( decoder_target_data.shape )

Max length of question is 16
Max length of answer is 19
(20867, 16)
(20867, 19)
(20867, 19, 4907)


In [3]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_tokens, 200 , mask_zero=True , weights=[embedding_matrix] ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_tokens, 200 , mask_zero=True, weights=[embedding_matrix]) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.load_weights('weights-friends_short-199-0.7454.hdf5')

model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 200)    981400      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    981400      input_2[0][0]                    
_____________________________________

In [4]:
filepath="weights-friends_short-SECOND-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

In [5]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1,  min_lr=0.000001, verbose=1,cooldown=1)
callbacks_list = [checkpoint, reduce_lr]

In [6]:
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=50, epochs=200, callbacks=callbacks_list ) 
model.save( 'model_friends_short-SECOND_epochs200.h5' ) 


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.7604
Epoch 00001: loss improved from inf to 0.76035, saving model to weights-friends_short-SECOND-01-0.7604.hdf5
20867/20867 [==============================] - 45s 2ms/sample - loss: 0.7604
Epoch 2/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.7534
Epoch 00002: loss improved from 0.76035 to 0.75365, saving model to weights-friends_short-SECOND-02-0.7537.hdf5
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.7537
Epoch 3/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.7492
Epoch 00003: loss improved from 0.75365 to 0.74917, saving model to weights-friends_short-SECOND-03-0.7492.hdf5
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.7492
Epoch 4/200
20850/20867 [============================>.] - ETA: 0

20850/20867 [============================>.] - ETA: 0s - loss: 0.5704
Epoch 00063: loss did not improve from 0.57032
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.5706
Epoch 64/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5705
Epoch 00064: loss did not improve from 0.57032
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.5706
Epoch 65/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5706
Epoch 00065: loss did not improve from 0.57032
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.5706
Epoch 66/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5711
Epoch 00066: loss did not improve from 0.57032
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.5711
Epoch 67/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5706
Epoch 00067: loss did not improve from 0.57032
20867/20867 [==============================] - 42s 2ms/

Epoch 102/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5707
Epoch 00102: loss did not improve from 0.57015
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.5707
Epoch 103/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5706
Epoch 00103: loss did not improve from 0.57015
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.5708
Epoch 104/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5706
Epoch 00104: loss did not improve from 0.57015
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.5705
Epoch 105/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5704
Epoch 00105: loss did not improve from 0.57015
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.5703
Epoch 106/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5707
Epoch 00106: loss did not improve from 0.57015
20867/20867 [========================

Epoch 141/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5706
Epoch 00141: loss did not improve from 0.56946
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.5707
Epoch 142/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5707
Epoch 00142: loss did not improve from 0.56946
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.5705
Epoch 143/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5708
Epoch 00143: loss did not improve from 0.56946
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.5708
Epoch 144/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5697
Epoch 00144: loss did not improve from 0.56946
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.5698
Epoch 145/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5701
Epoch 00145: loss did not improve from 0.56946
20867/20867 [========================

Epoch 180/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5705
Epoch 00180: loss did not improve from 0.56921
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.5704
Epoch 181/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5709
Epoch 00181: loss did not improve from 0.56921
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.5709
Epoch 182/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5694
Epoch 00182: loss did not improve from 0.56921
20867/20867 [==============================] - 42s 2ms/sample - loss: 0.5695
Epoch 183/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5694
Epoch 00183: loss did not improve from 0.56921
20867/20867 [==============================] - 43s 2ms/sample - loss: 0.5696
Epoch 184/200
20850/20867 [============================>.] - ETA: 0s - loss: 0.5701
Epoch 00184: loss did not improve from 0.56921
20867/20867 [========================

In [7]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [8]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        try:
            tokens_list.append( word_dict[ word ] ) 
        except:
            continue
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_question_len , padding='post')

In [10]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_answer_len:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

Enter question : how you doing
 yeah yeah i think it looks pretty good end
Enter question : how much
 all right i'm gonna go take a and take it's on the yeah end
Enter question : well
 yeah end
Enter question : i went out last night
 all right good think i'm is that i'm gotta get going with the are believe the guy you're your apartment
Enter question : how
 well i was know and i had a my something end
Enter question : stupid
 you got it end
Enter question : get out
 yeah end
Enter question : yeah what
 what end
Enter question : what what
 you guys were right end
Enter question : that's right! smart bot!
 oh can't you you know about end
